# Intro
Klik bij iedere cell op 'Run'. Controleer de uitvoer. Notebooks geven een stap-voor-stap uitvoering van code, waardoor gekeken kan worden of het goed verloopt.

In [1]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET

def to_xml(df, file_name=None, root_name='root', row_name='row', index_name='index', do_index=False, use_atts=False):
    root = ET.Element(root_name)
    tree = ET.ElementTree(root)
    for idx, row in df.iterrows():
        rowelm = ET.SubElement(root, row_name)
        if do_index:
            if use_atts:
                rowelm.set(index_name, str(idx))
            else:
                colelm = ET.SubElement(rowelm, index_name)
                colelm.text = str(idx)
        for hdr, val in row.iteritems():
            hdr = hdr.replace(' ', '_')
            if pd.notnull(val):
                val = str(val)
            else:
                val = ''
            if use_atts:
                rowelm.set(hdr, str(val))
            else:
                colelm = ET.SubElement(rowelm, hdr)
                colelm.text = str(val)
    if file_name is not None:
        tree.write(file_name, xml_declaration=False, encoding='utf8')
    return ET.tostring(root)

# Lees prerelease
De prerelease HTML tabel wordt van de LOINC site gelezen. Het kan even duren.

Controleer de output! Verwacht wordt een tabel zoals op de LOINC site te zien, als de LOINC HTML wijzigt kan dit niet goed gaan.

(Noot: de sumission queue heeft meer details, mogelijk kan dit ook een bron zijn.)

In [2]:
uri = 'https://loinc.org/prerelease/'
# uri = 'https://loinc.org/submissions/queue/'
prerelease = pd.read_html(uri)[0]
prerelease

,Created On,LOINC,Long Common Name,Special Use,Component,Prop,Time,System,Scale,Method,Shortname
0,2023-01-13,101406-7,Sudan virus RNA [Presence] in Blood by NAA wit...,Special Use codes are developed in response to...,Sudan virus RNA,PrThr,Pt,Bld,Ord,Probe.amp.tar,Sudan virus RNA Bld Ql NAA+probe
1,2023-01-10,101399-4,Monkeypox virus and Orthopoxvirus.non-variola ...,Special Use codes are developed in response to...,Monkeypox virus & Orthopoxvirus.non-variola DNA,Prid,Pt,XXX,Nom,Probe.amp.tar,MVPX & Nonvar Orthpx DNA Spec NAA+probe
2,2023-01-10,101398-6,Monkeypox virus and Orthopoxvirus.non-variola ...,Special Use codes are developed in response to...,Monkeypox virus & Orthopoxvirus.non-variola DN...,-,Pt,XXX,-,NaN,MVPX & Nonvar Orthpx DNA panel Spec
3,2023-01-09,101373-9,Francisella tularensis DNA [Presence] in Sputu...,Special Use codes are developed in response to...,Francisella tularensis DNA,PrThr,Pt,Sputum,Ord,Non-probe.amp.tar,F tular DNA Spt Ql NAA+non-probe
4,2023-01-09,101371-3,Yersinia pestis DNA [Presence] by NAA with non...,Special Use codes are developed in response to...,Yersinia pestis DNA,PrThr,Pt,Bld.pos growth,Ord,Non-probe.amp.tar,Y pestis DNA Bld Pos Ql NAA+non-probe
...,...,...,...,...,...,...,...,...,...,...,...
579,2022-08-23,100971-1,Clinical record panel,NaN,Clinical record panel,-,Pt,^Patient,-,NaN,NaN
580,2022-08-15,100970-3,Portable medical order completion information,NaN,Portable medical order completion information,-,-,^Patient,-,POLST,NaN
581,2022-08-11,100969-5,Pathology report additional specified observat...,NaN,Pathology report.additional specified observation,Find,Pt,Specimen,Nar,NaN,NaN
582,2022-01-13,100001-7,Health informatics pioneer and cofounder of LOINC,NaN,Health informatics pioneer and cofounder of LOINC,Hx,Pt,^Patient,Nar,NaN,NaN


# Formatteren van de headers

In [3]:
prerelease.drop(['Created On', 'Special Use'], axis=1, inplace=True)
prerelease.columns = ['LOINC', 'longName', 'component', 'property', 'timing', 'system', 'scale', 'method', 'shortName']
prerelease.head()

,LOINC,longName,component,property,timing,system,scale,method,shortName
0,101406-7,Sudan virus RNA [Presence] in Blood by NAA wit...,Sudan virus RNA,PrThr,Pt,Bld,Ord,Probe.amp.tar,Sudan virus RNA Bld Ql NAA+probe
1,101399-4,Monkeypox virus and Orthopoxvirus.non-variola ...,Monkeypox virus & Orthopoxvirus.non-variola DNA,Prid,Pt,XXX,Nom,Probe.amp.tar,MVPX & Nonvar Orthpx DNA Spec NAA+probe
2,101398-6,Monkeypox virus and Orthopoxvirus.non-variola ...,Monkeypox virus & Orthopoxvirus.non-variola DN...,-,Pt,XXX,-,NaN,MVPX & Nonvar Orthpx DNA panel Spec
3,101373-9,Francisella tularensis DNA [Presence] in Sputu...,Francisella tularensis DNA,PrThr,Pt,Sputum,Ord,Non-probe.amp.tar,F tular DNA Spt Ql NAA+non-probe
4,101371-3,Yersinia pestis DNA [Presence] by NAA with non...,Yersinia pestis DNA,PrThr,Pt,Bld.pos growth,Ord,Non-probe.amp.tar,Y pestis DNA Bld Pos Ql NAA+non-probe


# Maak XML
Het dataframe wordt geëxporteerd naar XML.
Vul target, usernaam, password in. Let op: eerst de cell runnen, dan waarden invullen. Cell niet opnieuw runnen, dat zet de default waarden terug.

In [7]:
xml = to_xml(prerelease, 'prerelease.xml', root_name='prerelease', row_name='concept')

In [8]:
import ipywidgets as widgets
targeturi = widgets.Dropdown(
    options=[('Productie', 'https://labterminologie.nl/lab/prerelease'),
             ('Acceptatie', 'https://terminologie.test-nictiz.nl/lab/prerelease'),
             ('8877', 'http://localhost:8877/exist/apps/lab/prerelease')],
    description='Target:',
)
usr = widgets.Text(description='Username')
pwd = widgets.Password(description='Password')
widgets.VBox([
  targeturi, usr, pwd
])

In [15]:
targeturi.value

'https://labterminologie.nl/lab/prerelease'


# Upload naar de server
Verwachte uitvoer: (HTTP status) 201, dat is 'Created'

In [16]:
resp = requests.put(targeturi.value, data=xml, auth=requests.auth.HTTPBasicAuth(usr.value, pwd.value))
pwd.value = ''
resp.status_code

201

# Controleer de uitvoer
Onderstaande links voeren naar de prerelease tabel zoals zojuist geüpload.

[LOINC prerelease](https://loinc.org/prerelease/)

[prerelease productie](http://labterminologie.nl/lab/prerelease)

[prerelease acceptatie](https://terminologie.test-nictiz.nl/lab/prerelease)

[prerelease lokaal 8877](http://localhost:8877/exist/apps/lab-data/data/prerelease.xml)